In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
import pandas as pd
from ast import literal_eval
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, BertModel, RobertaForSequenceClassification, AutoModelForSequenceClassification
from transformers import AutoTokenizer, BertTokenizer, BertTokenizerFast, BertConfig, RobertaTokenizer
from datasets import Dataset
import torch
import numpy as np
from scipy.special import softmax, expit

import os
#os.environ['CUDA_VISIBLE_DEVICES']='1'

In [ ]:
import random

seed_val = 42

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

setup_seed(seed_val)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp -av /content/drive/MyDrive/CUHK/ling6920/yc-data-4-23-2 /content/data

In [ ]:
def read_file(file):
    with open(file) as f:
        lines = f.readlines()
    return lines

In [ ]:
def write_output(file, str_in):
    with open(file, 'w') as f:
        f.write(str_in)

In [ ]:
list_file_name = ['/content/data/train.txt', '/content/data/dev.txt', '/content/data/test.txt']
dict_sent_list = dict()
for file in list_file_name: 
    list_sent = read_file(file)
    new_list_sent = ['sentence\tlabel']
    for sent in list_sent:
        sent = sent.strip()
        if '[SEP]' in sent and '[AGE=' in sent:
            split_sep = sent.split('[SEP]')[0]
            split_age = split_sep.split('[AGE=')
            for seq_age in split_age:
                if ']' in seq_age:
                    split_seq_age = seq_age.split(']')
                    age_info = split_seq_age[0]
                    text_info = ']'.join(split_seq_age[1:]).replace('\t', ' ').strip()
                    new_list_sent.append(text_info + '\t' + age_info)
                #else:
                #    print(seq_age)
    dict_sent_list[file] = new_list_sent
    write_output(file.split('.txt')[0] + '_label.tsv', '\n'.join(new_list_sent))

In [ ]:
train_df_voc = pd.read_csv('data/train_label.tsv', sep='\t')[['sentence', 'label']]
test_df_voc = pd.read_csv('data/test_label.tsv', sep='\t')[['sentence', 'label']]
valid_df_voc = pd.read_csv('data/dev_label.tsv', sep='\t')[['sentence', 'label']]

In [ ]:
train_df_voc.head()

In [ ]:
train_df_voc['label'].value_counts()

In [ ]:
def change_label(df):
    for i, row in df.iterrows():
        ifor_val = df.loc[i,'Intensity Class'].split(':')[0]
        if int(ifor_val) < 0:
            ifor_val = 1  # neg
        elif int(ifor_val) > 0:
            ifor_val = 2  # pos
        else: 
            ifor_val = 0  # neu
        df.at[i,'Intensity Class'] = ifor_val
    return df

In [ ]:
def make_dataset(df, tokenizer):
  dataset_train = Dataset.from_pandas(df)
  dataset_train = dataset_train.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=128), batched=True)
  dataset_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
  return dataset_train

In [ ]:
def train_model(train_df, valid_df, model_name, dir_model, num_labels):
  print('-----train-----')

  if model_name == 'bert':
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  elif model_name == 'roberta':
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
  elif model_name == 'bertweet':
    tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

  dataset_train = make_dataset(train_df, tokenizer)
  dataset_val = make_dataset(valid_df, tokenizer)

  training_args = TrainingArguments(
                  output_dir='./'+dir_model,          # output directory
                  num_train_epochs=8,              # total # of training epochs
                  per_device_train_batch_size=32,  # batch size per device during training
                  per_device_eval_batch_size=16,   # batch size for evaluation
                  warmup_steps=500,                # number of warmup steps for learning rate scheduler
                  weight_decay=0.01,               # strength of weight decay
                  logging_dir='./logs_'+dir_model,            # directory for storing logs
                  #evaluation_strategy="steps",
                  evaluation_strategy="epoch",
                  save_strategy="epoch",
                  load_best_model_at_end = True,
                  seed=seed_val,
                  overwrite_output_dir=True,
  )

  if model_name == 'bert':
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)
  elif model_name == 'roberta':
    model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)
  elif model_name == 'bertweet':
    model = AutoModelForSequenceClassification.from_pretrained('vinai/bertweet-base', num_labels=num_labels)

  #model = model.to(device)
  model.train()

  trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=dataset_train,         # training dataset
      eval_dataset=dataset_val,            # evaluation dataset
  )

  trainer.train()
  
  return tokenizer, trainer


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, precision_recall_fscore_support

In [ ]:
def eval_model(trainer, test_df, tokenizer, overall_types):
  print('-----eval-----')
  dataset_test = make_dataset(test_df, tokenizer)
  predict_data = trainer.predict(dataset_test)
  metrics = predict_data.metrics
  print(metrics)
  #print(np.argmax(predict_data.predictions, axis=1).flatten())
  pre_labels = np.argmax(predict_data.predictions, axis=1).flatten()
  test_df['predictions'] = pd.Series(pre_labels)

  #write_out(overall_types, metrics)
  y_true = test_df['label'].values.tolist()
  y_pred = test_df['predictions'].values.tolist()
  report = classification_report(y_true, y_pred)
  print(report)

  precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
  acc = accuracy_score(y_true, y_pred)
  print('precision: ' + str(precision))
  print('recall: ' + str(recall))
  print('f1: ' + str(f1))
  print('accuracy: ' + str(acc))

  return metrics, test_df

In [ ]:
num_labels = 8

In [ ]:
# voc, bert

tokenizer, trainer_voc = train_model(train_df_voc, valid_df_voc, 'bert', 'voc-bert', num_labels)

metrics_voc, pred_df_voc = eval_model(trainer_voc, test_df_voc, tokenizer, 'voc-bert')
pred_df_voc.to_csv('pred_df_voc-bert.csv', index=False)

trainer_voc.save_model('./model_save_voc-bert')